In [1]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ee
import geemap
import geopandas as gpd


In [2]:
from constants import   STUDY_BOUNDARY_PATH


# Import Landsat8 OLI

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#terms-of-use

 Landsat-8 image courtesy of the U.S. Geological Survey

In [3]:
#ee.Authenticate()
ee.Initialize()

In [4]:
# Create an interactive map
Map = geemap.Map()



In [5]:
# Display the map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# Define the collection
collection = "LANDSAT/LC08/C02/T1_L2"

In [7]:
shapefile_path = STUDY_BOUNDARY_PATH
study_boundary = gpd.read_file(shapefile_path)

In [8]:
# Load the study area
ee_boundary = geemap.geopandas_to_ee(study_boundary)


In [9]:
aoi = ee_boundary.geometry()

In [10]:
# Define the dates of interest
dates = ['2021-07-06', '2021-07-22', '2021-08-07', '2021-10-10', '2021-10-26', '2021-11-11']


In [11]:
image = ee.ImageCollection(collection) \
    .filterDate('2021-07-06', '2021-07-07') \
    .filterBounds(aoi) \
    .first()  # get the first image that matches the filters

In [12]:
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    image = image.addBands(optical_bands, None, True)
    image = image.addBands(thermal_bands, None, True)
    return image


In [13]:
vis_params = { "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
  "min": 0.0,
  "max": 0.3
}

In [14]:
# Loop through the dates and get the imagery
for date in dates:
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')

    # Filter the image collection by date and area
    image = ee.ImageCollection(collection) \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi) \
        .first()  # get the first image that matches the filters

    if image:  # check if image exists
        print(f"Image for date {date} found!")
        processed_image = apply_scale_factors(image)
        Map.addLayer(processed_image, vis_params, date)  # add the image to the map
    else:
        print(f"No image found for date {date}")

# Set the map to focus on the study area
Map.centerObject(ee_boundary, zoom=10)



Image for date 2021-07-06 found!
Image for date 2021-07-22 found!
Image for date 2021-08-07 found!
Image for date 2021-10-10 found!
Image for date 2021-10-26 found!
Image for date 2021-11-11 found!


In [16]:
chloro_map = geemap.Map()
chloro_map.centerObject(ee_boundary, zoom=10)
chloro_map

Map(center=[33.90594544379043, -118.40630988275745], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
def trinh_et_al_chl_a(image):
    a_0 = 0.9375
    a_1 = -1.8862
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    image = image.addBands(optical_bands, None, True)
    image = image.addBands(thermal_bands, None, True)
    return image

In [ ]:
# Loop through the dates and get the imagery
for date in dates:
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')

    # Filter the image collection by date and area
    image = ee.ImageCollection(collection) \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi) \
        .first()  # get the first image that matches the filters

    if image:  # check if image exists
        print(f"Image for date {date} found!")
        processed_image = apply_scale_factors(image)
        Map.addLayer(processed_image, vis_params, date)  # add the image to the map
    else:
        print(f"No image found for date {date}")

# Set the map to focus on the study area
Map.centerObject(ee_boundary, zoom=10)